In [40]:
import os
import os.path as op
import openneuro

from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

In [41]:
dataset = 'ds003685'

# Download one subject's data from each dataset
bids_root = op.join(op.dirname(sample.data_path()), dataset)
if not op.isdir(bids_root):
    os.makedirs(bids_root)

openneuro.download(dataset=dataset, target_dir=bids_root,
                   include=['sub-01*eyes*'])


👋 Hello! This is openneuro-py 2022.1.0. Great to see you! 🤗

   👉 Please report problems 🤯 and bugs 🪲 at
      https://github.com/hoechenberger/openneuro-py/issues

🌍 Preparing to download ds003685 …
👉 Retrieving up to 40 files (5 concurrent downloads).
✅ Finished downloading ds003685.

🧠 Please enjoy your brains.



In [42]:
print_dir_tree(bids_root, max_depth=4)

|ds003685/
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.tsv
|--- sub-01/
|------ ses-session1/
|--------- eeg/
|------------ sub-01_ses-session1_task-eyes open_run-2_channels.tsv
|------------ sub-01_ses-session1_task-eyes open_run-2_eeg.eeg
|------------ sub-01_ses-session1_task-eyes open_run-2_eeg.json
|------------ sub-01_ses-session1_task-eyes open_run-2_eeg.vhdr
|------------ sub-01_ses-session1_task-eyes open_run-2_eeg.vmrk
|------------ sub-01_ses-session1_task-eyes open_run-2_events.tsv
|------------ sub-01_ses-session1_task-eyesclosed_run-1_channels.tsv
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.eeg
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.json
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.vhdr
|------------ sub-01_ses-session1_task-eyesclosed_run-1_eeg.vmrk
|------------ sub-01_ses-session1_task-eyesclosed_run-1_events.tsv
|------ ses-session2/
|--------- eeg/
|------------ sub-01_ses-session2_task-

In [43]:
print(make_report(bids_root))

Summarizing participants.tsv /Users/pellet/mne_data/ds003685/participants.tsv...
The participant template found: comprised of 28 male and 32 female participants;
handedness were all unknown;
ages ranged from 18.0 to 28.0 (mean = 20.0, std = 1.89; 5 with unknown age)
 The A test-retest resting and cognitive state EEG dataset dataset was created
by Yulin Wang, Wei Duan, Lihong Ding, Debo Dong, and Xu Lei and conforms to BIDS
version 1.2. This report was generated with MNE-BIDS
(https://doi.org/10.21105/joss.01896). The dataset consists of 9 participants
(comprised of 28 male and 32 female participants; handedness were all unknown;
ages ranged from 18.0 to 28.0 (mean = 20.0, std = 1.89; 5 with unknown age)) and
3 recording sessions: session1, session2, and session3.


/var/folders/23/fzc0x7wn53q2m_2n53dlm5xr0000gn/T/ipykernel_18750/3390368180.py:1: RuntimeWarning: No *scans.tsv files found. Currently, we do not generate a report without the scans.tsv files.
  print(make_report(bids_root))


In [44]:
import re

str = 'ds003685/sub-01/ses-session2/eeg/sub-01_ses-session2_task-eyesopen_run-2_eeg.vmrk'

print(re.sub(r'eyesopen', r'eyes open', str))

ds003685/sub-01/ses-session2/eeg/sub-01_ses-session2_task-eyes open_run-2_eeg.vmrk


In [45]:
#fix files missing spaces

import re, glob, os

def renamer(files, pattern, replacement):
    for pathname in glob.glob(files, recursive=True):
        new_filename = re.sub(pattern, replacement, pathname)
        if new_filename != pathname:
            os.rename(
              pathname,
              os.path.join(os.path.dirname(pathname), new_filename))
            print('renamed to {}'.format(os.path.join(os.path.dirname(pathname), new_filename)))

renamer('{}/**/*'.format(bids_root), 'eyesopen', 'eyes open')
renamer('{}/**/*'.format(bids_root), 'eyesclosed', 'eyes closed')

/Users/pellet/mne_data/ds003685/CHANGES
/Users/pellet/mne_data/ds003685/sub-08
/Users/pellet/mne_data/ds003685/sub-01
/Users/pellet/mne_data/ds003685/sub-06
/Users/pellet/mne_data/ds003685/sub-07
/Users/pellet/mne_data/ds003685/sub-09
/Users/pellet/mne_data/ds003685/README
/Users/pellet/mne_data/ds003685/dataset_description.json
/Users/pellet/mne_data/ds003685/participants.tsv
/Users/pellet/mne_data/ds003685/sub-05
/Users/pellet/mne_data/ds003685/sub-02
/Users/pellet/mne_data/ds003685/sub-03
/Users/pellet/mne_data/ds003685/sub-04
/Users/pellet/mne_data/ds003685/sub-08/ses-session1
/Users/pellet/mne_data/ds003685/sub-08/ses-session2
/Users/pellet/mne_data/ds003685/sub-08/ses-session3
/Users/pellet/mne_data/ds003685/sub-08/ses-session1/eeg
/Users/pellet/mne_data/ds003685/sub-08/ses-session1/eeg/sub-08_ses-session1_task-eyes open_run-2_eeg.eeg
/Users/pellet/mne_data/ds003685/sub-08/ses-session1/eeg/sub-08_ses-session1_task-eyes open_run-2_events.tsv
/Users/pellet/mne_data/ds003685/sub-08/

In [46]:
print(bids_root)
datatype = 'eeg'
suffix = 'eeg'
bids_paths = [ 
    BIDSPath(task='eyes open', suffix=suffix, datatype=datatype, root=bids_root),
    BIDSPath(task='eyes closed', suffix=suffix, datatype=datatype, root=bids_root)]


/Users/pellet/mne_data/ds003685


In [53]:
from tqdm import tqdm

def get_vhdr_files(bids_path):
    # get names of the .vhdr files
    basenames = []
    match = bids_path.match()
    for i in tqdm(range(0, len(match))):
        if match[i].basename[-4:] == 'vhdr':
            basenames.append(match[i])
    return basenames

vhdr_files = []
for bids_path in bids_paths:
    vhdr_files.append(get_vhdr_files(bids_path))

100%|██████████| 9/9 [00:00<00:00, 65196.44it/s]


In [55]:

def get_raw_files(basenames):
    raws = []
    for basename in tqdm(basenames):
        #print(basename)
        raw = read_raw_bids(bids_path=basename, verbose=False)
        raw.load_data()
        #print(raw.ch_names)
        raw = raw.pick_channels(['Oz'])
        raw = raw.filter(1,40)
        raw = raw.resample(100, npad="auto")
        raws.append(raw)
    
#print(vhdr_files)
open_eyes = get_raw_files(vhdr_files[0])
closed_eyes = get_raw_files(vhdr_files[1])


  0%|          | 0/27 [00:00<?, ?it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 sec)



  4%|▎         | 1/27 [00:00<00:03,  6.90it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 sec)



  7%|▋         | 2/27 [00:00<00:03,  7.50it/s]

...

 33%|███▎      | 1/3 [00:00<00:00,  8.30it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 sec)



 67%|██████▋   | 2/3 [00:00<00:00,  8.64it/s]

Reading 0 ... 149999  =      0.000 ...   299.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 sec)



100%|██████████| 3/3 [00:00<00:00,  8.67it/s]


In [ ]:
#load all raw array into one raw file to run eeg_power_band 
#function over and compare difference between eyes open and closed with alpha band power.
